# Data Visualization

This notebook is just going to be used for visualizing the radar data so that we can see how the data is structured for further use

In [1]:
## Imports

import bz2
import sys, os
import pyart
import matplotlib.pyplot as plt
from pyart.graph import RadarDisplay



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [4]:
# Defining the data file and opening the data
# Define the path to the data file
root_dir ='/mnt/d/VS-workspace/personal_projects/storm_chasing_app/'
data_path = 'data/'
file = "KTLX_latest_scan.bin"  # Replace with your file name
file_path = f"{root_dir}{data_path}{file}"
file_path = os.path.join(root_dir, data_path, file)

# Try to read the file using Py-ART
try:
    radar = pyart.io.read_nexrad_archive(file_path)
    print("Radar data loaded successfully.")
    print(radar)  # Print radar object details

    # Example: Access radar attributes
    print("Radar fields:", radar.fields.keys())
    print("Radar elevation angles:", radar.elevation['data'])
except Exception as e:
    print(f"Error reading radar file: {e}")


Radar data loaded successfully.
Radar fields: dict_keys(['differential_phase', 'reflectivity', 'velocity', 'spectrum_width', 'cross_correlation_ratio', 'differential_reflectivity', 'clutter_filter_power_removed'])
Radar elevation angles: [0.48614502 0.35980225 0.42022705 ... 0.83496094 0.83496094 0.83496094]


In [11]:
type(radar)

pyart.core.radar.Radar